<img src="https://i.imgur.com/Y6EMKKg.jpg" style="float: left; margin: 15px;" width="75">


# Data Cleaning for EDA

**Author: Joseph Darby**

---


## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 200

%matplotlib inline

### Function to Create a Column with the Match Result

In [2]:
def get_match_result(df):
    
    result_list = []
    
    for i in range(df.shape[0]):
        home_score = df['home_team_goal'].iloc[i]
        away_score = df['away_team_goal'].iloc[i]
        if home_score > away_score:
            result_list.append("W")
        elif home_score < away_score:
            result_list.append("L")
        else:
            result_list.append("D")
    df['match_result'] = result_list
    
    return "Success!"

## Import Match CSV and Create Dataframe

In [3]:
# Dataframe of the "Match", "Team", "League", and "Country" tables joined together
combined_df = pd.read_csv('./datasets_database/combined_tables.csv')

# Add target vector columns that of the results of each match (Home-Win = 'W', Draw = "D", Away-Win = "L")
get_match_result(combined_df)

'Success!'

-----
## Parsed Dataframe

In [4]:
# Make a list of the features we are interested in modeling & exploring
a_parsed_col_list = ['country','league_name', 'home_team', 'away_team', 'season', 'year_month', 'home_team_goal', 
                   'away_team_goal','match_result', 'match_api_id', 'stage', 'home_team_api_id', 'away_team_api_id', 
                   'B365A', 'B365D', 'B365H', 'BWA', 'BWD', 'BWH', 'IWA', 'IWD', 'IWH', 'away_player_1', 'home_player_1',
                   'away_player_2', 'home_player_2','away_player_3', 'home_player_3','away_player_4', 'home_player_4',
                   'away_player_5', 'home_player_5','away_player_6', 'home_player_6','away_player_7', 'home_player_7',
                   'away_player_8', 'home_player_8','away_player_9', 'home_player_9', 'away_player_10', 'home_player_10',
                   'away_player_11', 'home_player_11']


# Create Dataframe of columns we are interested in & drop null observations
parsed_df = combined_df[a_parsed_col_list].dropna()

In [5]:
# The season year needs to be one year instead of a combined string in order to join on team and player attributes
parsed_df['year'] = parsed_df['season'].map(lambda x: int(x[5:]))

replace_dict = {2009: 2010, 2010: 2010, 2011: 2011, 2012: 2012, 2013: 2013, 2014: 2014, 2015: 2015, 2016: 2015}

parsed_df['year'] = parsed_df['year'].map(replace_dict)

parsed_df.drop(columns='season', inplace=True)

In [6]:
# Home & Away_Player data types need to match that of the Player Attributes table
col_dict = {}
col_list = ['year', 'away_player_1', 'home_player_1','away_player_2', 'home_player_2','away_player_3', 
             'home_player_3','away_player_4', 'home_player_4','away_player_5', 'home_player_5', 'away_player_6', 
             'home_player_6','away_player_7', 'home_player_7','away_player_8', 'home_player_8','away_player_9', 
             'home_player_9', 'away_player_10', 'home_player_10','away_player_11', 'home_player_11']

for each in col_list:
    col_dict[each] = 'int64'
    
parsed_df = parsed_df.astype(col_dict)
parsed_df.reset_index(drop = True, inplace = True)

In [7]:
# Drop 'season' series and re-configure series layout
b_parsed_col_list = ['country','league_name', 'home_team', 'away_team', 'year', 'year_month', 'home_team_goal', 
                   'away_team_goal','match_result', 'match_api_id', 'stage', 'home_team_api_id', 'away_team_api_id', 
                   'B365A', 'B365D', 'B365H', 'BWA', 'BWD', 'BWH', 'IWA', 'IWD', 'IWH', 'away_player_1', 'home_player_1',
                   'away_player_2', 'home_player_2','away_player_3', 'home_player_3','away_player_4', 'home_player_4',
                   'away_player_5', 'home_player_5','away_player_6', 'home_player_6','away_player_7', 'home_player_7',
                   'away_player_8', 'home_player_8','away_player_9', 'home_player_9', 'away_player_10', 'home_player_10',
                   'away_player_11', 'home_player_11']

# Create Dataframe of columns we are interested in & drop null observations
parsed_df = parsed_df[b_parsed_col_list]

In [8]:
parsed_df.shape

(19620, 44)

---
# Incorporate Team Attributes

## Created a Function:

- Imports Team Attributes
- Removes unneccssary columns
- Reorganizes column layout
- **Removes rows where there are multiple listings per team team by year**

In [9]:
def create_clean_df(path, prefix):
    # Team Attributes dataframe to merge & drop unnecessary columns
    df = pd.read_csv(path)
    df.drop(columns = ['buildUpPlayDribbling', 'team_fifa_api_id'], inplace = True)
    
    # Create 'date_year' column and drop 'date'
    df['year'] = df['year_month'].map(lambda x: int(x[:4]))
    
    # Change column order & reset index after sort
    team_col_order = ['home_team_api_id', 'home_year', 'home_year_month','home_buildUpPlaySpeed', 'home_buildUpPlaySpeedClass', 
                      'home_buildUpPlayDribblingClass','home_buildUpPlayPassing', 'home_buildUpPlayPassingClass',
                      'home_buildUpPlayPositioningClass','home_chanceCreationPassing', 'home_chanceCreationPassingClass',
                      'home_chanceCreationCrossing','home_chanceCreationCrossingClass', 'home_chanceCreationShooting',
                      'home_chanceCreationShootingClass','home_chanceCreationPositioningClass', 'home_defencePressure',
                      'home_defencePressureClass','home_defenceAggression', 'home_defenceAggressionClass',
                      'home_defenceTeamWidth','home_defenceTeamWidthClass', 'home_defenceDefenderLineClass']
    new_order = [each.replace('home_','') for each in team_col_order]
    df = df[new_order]
    df = df.sort_values(by= ['team_api_id', 'year'], ascending=[True, True]).reset_index(drop=True)
    
    # Remove rows where a team has multiple rows for the same year
    # Create a set of team_id's
    api_id_set = set(df['team_api_id'].values)

    # Create a list of team_id's and all years on record
    all_team_years = []
    for team_id in api_id_set:
        team_years = df[df.loc[:, 'team_api_id'] == team_id]['year']
        for record in team_years:
            all_team_years.append(f'{team_id}, {record}')

    # Create a set indicating team and duplicated year
    all_duplicates = []
    for each_listing in all_team_years:
        if all_team_years.count(each_listing) > 1:
            all_duplicates.append(each_listing)
        else:
            continue
    all_duplicates = list(set(all_duplicates))
    
    # Keep one row but remove the other duplicated year rows per team in the set
    for each in all_duplicates:
        indexes_to_drop = df[(df['team_api_id'] == int(each[0:4])) & (df['year'] == int(each[6:]))].index[1:]
        df.drop(indexes_to_drop, inplace = True)

    # add 'home_' or 'away_'
    df = df.add_prefix(prefix)    
    return df

## Home Team Dataframe

In [21]:
# Dataframe of the "Match", "Team", "League", and "Country" tables joined together
home_team_df = pd.read_csv('./datasets_database/team_attributes.csv')

# Use custom function on Home Team data
home_team_df = create_clean_df('./datasets_database/team_attributes.csv', 'home_')

print(home_team_df.shape)
home_team_df.head()

(1457, 23)


,home_team_api_id,home_year,home_year_month,home_buildUpPlaySpeed,home_buildUpPlaySpeedClass,home_buildUpPlayDribblingClass,home_buildUpPlayPassing,home_buildUpPlayPassingClass,home_buildUpPlayPositioningClass,home_chanceCreationPassing,home_chanceCreationPassingClass,home_chanceCreationCrossing,home_chanceCreationCrossingClass,home_chanceCreationShooting,home_chanceCreationShootingClass,home_chanceCreationPositioningClass,home_defencePressure,home_defencePressureClass,home_defenceAggression,home_defenceAggressionClass,home_defenceTeamWidth,home_defenceTeamWidthClass,home_defenceDefenderLineClass
0,1601,2010,2010-02-01,30,Slow,Little,40,Mixed,Organised,50,Normal,35,Normal,70,Lots,Organised,65,Medium,60,Press,50,Normal,Cover
1,1601,2011,2011-02-01,48,Balanced,Little,51,Mixed,Organised,68,Risky,67,Lots,51,Normal,Organised,46,Medium,48,Press,50,Normal,Cover
2,1601,2012,2012-02-01,53,Balanced,Little,55,Mixed,Organised,44,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover
3,1601,2013,2013-09-01,53,Balanced,Little,55,Mixed,Organised,44,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover
4,1601,2014,2014-09-01,53,Balanced,Normal,38,Mixed,Organised,66,Normal,65,Normal,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover


## Away Team Dataframe

In [13]:
# Use custom function on Home Team data
away_team_df = create_clean_df('./datasets_database/team_attributes.csv', 'away_')

print(away_team_df.shape)
away_team_df.head()

### Merge all three together

In [15]:
parsed_df = pd.merge(parsed_df, home_team_df, left_on=['home_team_api_id', 'year'], right_on=['home_team_api_id', 'home_year'], how='left')
parsed_df = pd.merge(parsed_df, away_team_df, left_on=['away_team_api_id', 'year'], right_on=['away_team_api_id', 'away_year'], how='left')

# Player Attributes

I read in the player_attributes.csv and mapped the names and the player's average overall Fifa rating to the merged dataframe.

In [16]:
# Import CSV
player_attr_df = pd.read_csv('./datasets_database/player_attributes.csv')

In [17]:
# Create a dictionary to map player names
num_id = list(player_attr_df["player_api_id"].values)
player_name = list(player_attr_df['player_name'].values)
player_dict = dict(zip(num_id, player_name))

In [18]:
# Create a dictionary to map each player's average overall rating
avg_df = player_attr_df.groupby(by='player_api_id').mean()
overall_rate_dict = dict(avg_df.overall_rating)

# Create a list of the series to map over
player_series_list = ['away_player_1', 'home_player_1', 'away_player_2', 'home_player_2', 'away_player_3',
                      'home_player_3', 'away_player_4', 'home_player_4', 'away_player_5', 'home_player_5', 
                      'away_player_6', 'home_player_6', 'away_player_7', 'home_player_7', 'away_player_8', 
                      'home_player_8', 'away_player_9', 'home_player_9', 'away_player_10', 'home_player_10', 
                      'away_player_11', 'home_player_11']

# Create a for loop to map names and create overall rating columns
for each in player_series_list:
    parsed_df[each + '_rating'] = parsed_df[each].map(overall_rate_dict)
    parsed_df[each] = parsed_df[each].map(player_dict)

In [19]:
# Re-configure column layout and perge columns that are no longer needed
col = ['country', 'league_name', 'home_team', 'away_team', 'year', 'year_month', 'home_team_goal', 'away_team_goal', 
       'match_result', 'B365A', 'B365D', 'B365H', 'BWA', 'BWD', 'BWH', 'IWA', 'IWD', 'IWH', 'home_player_1', 
       'home_player_1_rating', 'home_player_2', 'home_player_2_rating','home_player_3', 'home_player_3_rating',
       'home_player_4', 'home_player_4_rating', 'home_player_5','home_player_5_rating', 'home_player_6', 
       'home_player_6_rating', 'home_player_7', 'home_player_7_rating', 'home_player_8', 'home_player_8_rating', 
       'home_player_9', 'home_player_9_rating', 'home_player_10', 'home_player_10_rating', 'home_player_11',
       'home_player_11_rating', 'away_player_1', 'away_player_1_rating','away_player_2', 'away_player_2_rating', 
       'away_player_3', 'away_player_3_rating', 'away_player_4', 'away_player_4_rating', 'away_player_5',
       'away_player_5_rating', 'away_player_6', 'away_player_6_rating','away_player_7', 'away_player_7_rating',
       'away_player_8', 'away_player_8_rating', 'away_player_9', 
       'away_player_9_rating', 'away_player_10', 'away_player_10_rating', 'away_player_11', 'away_player_11_rating',
       'home_buildUpPlaySpeed', 'home_buildUpPlaySpeedClass', 'home_buildUpPlayDribblingClass',
       'home_buildUpPlayPassing', 'home_buildUpPlayPassingClass', 'home_buildUpPlayPositioningClass',
       'home_chanceCreationPassing', 'home_chanceCreationPassingClass', 'home_chanceCreationCrossing',
       'home_chanceCreationCrossingClass', 'home_chanceCreationShooting', 'home_chanceCreationShootingClass',
       'home_chanceCreationPositioningClass', 'home_defencePressure', 'home_defencePressureClass', 'home_defenceAggression',
       'home_defenceAggressionClass', 'home_defenceTeamWidth', 'home_defenceTeamWidthClass', 'home_defenceDefenderLineClass',
       'away_buildUpPlaySpeed', 'away_buildUpPlaySpeedClass', 'away_buildUpPlayDribblingClass', 
       'away_buildUpPlayPassing', 'away_buildUpPlayPassingClass', 'away_buildUpPlayPositioningClass',
       'away_chanceCreationPassing', 'away_chanceCreationPassingClass', 'away_chanceCreationCrossing',
       'away_chanceCreationCrossingClass', 'away_chanceCreationShooting', 'away_chanceCreationShootingClass', 
       'away_chanceCreationPositioningClass', 'away_defencePressure', 'away_defencePressureClass', 'away_defenceAggression',
       'away_defenceAggressionClass', 'away_defenceTeamWidth', 'away_defenceTeamWidthClass', 'away_defenceDefenderLineClass']

In [20]:
# Assign new layout to a new dataframe and save dataframe to a csv for EDA and modeling
parsed_df = parsed_df[col]
parsed_df.to_csv('./datasets_database/6.11_combined_df.csv')